## Step - 1 : Collect the Data from JSON into the Pandas for Visibility 

In [190]:
import os
import pandas as pd
import json
import re
from rouge_score import rouge_scorer
from statistics import mean
import numpy as np

In [191]:
canonical_dev = os.listdir(os.path.join(os.path.abspath(os.getcwd()),"./liputan6_data/canonical/dev"))
canonical_test = os.listdir(os.path.join(os.path.abspath(os.getcwd()),"./liputan6_data/canonical/test"))
canonical_train = os.listdir(os.path.join(os.path.abspath(os.getcwd()),"./liputan6_data/canonical/train"))

In [192]:
clean_article = []
clean_summary = []
extractive_summary = []

In [ ]:
for fileJSON in canonical_dev : 
    with open(os.path.join(os.path.abspath(os.getcwd()),"./liputan6_data/canonical/dev",fileJSON),"r") as f : 
        file = json.load(f)
        sentence_result = "" 
        for sentence in file['clean_article'] : 
            sentence_result += " ".join(sentence)
            sentence_result += " "
        sentence_result = sentence_result.split(" : ",maxsplit = 2)[-1]
        sentence_result = re.sub(pattern = r"\s\.\s", repl = ". ", string = sentence_result)
        sentence_result = re.sub(pattern = r"\s,\s", repl = ", ", string = sentence_result)
        sentence_result = re.sub(pattern = r"\s\? ", repl = "? ", string = sentence_result)
        sentence_result = re.sub(pattern = r"\s!\s", repl = "! ", string = sentence_result)
        sentence_result = re.sub(r"\s\(\s", r" \(", sentence_result)
        sentence_result = re.sub(r"\s\)\s", r"\)", sentence_result)
        sentence_result = re.sub(r"\\\(", "(", sentence_result)
        sentence_result = re.sub(r"\\\)", ")", sentence_result)
        clean_article.append(sentence_result.lower())
        sentence_result = "" 
        for sentence in file['clean_summary'] : 
            sentence_result+=" ".join(sentence)
            sentence_result+=" "
        sentence_result = re.sub(pattern = r"\s\.\s", repl = ". ", string = sentence_result)
        sentence_result = re.sub(pattern = r"\s,\s", repl = ", ", string = sentence_result)
        sentence_result = re.sub(pattern = r"\s\? ", repl = "? ", string = sentence_result)
        sentence_result = re.sub(pattern = r"\s!\s", repl = "! ", string = sentence_result)
        sentence_result = re.sub(r"\s\(\s", r" \(", sentence_result)
        sentence_result = re.sub(r"\s\)\s", r"\)", sentence_result)
        sentence_result = re.sub(r"\\\(", "(", sentence_result)
        sentence_result = re.sub(r"\\\)", ")", sentence_result)
        clean_summary.append(sentence_result.lower())


In [193]:
clean_article_list = []
clean_summary_list = []
kunci_jawaban = []
for fileJSON in canonical_dev : 
    with open(os.path.join(os.path.abspath(os.getcwd()),"./liputan6_data/canonical/dev",fileJSON),"r") as f : 
        file = json.load(f)
        clean_article_list.append([])
        clean_summary_list.append([])
        sentence_result = "" 
        for sentence in file['clean_article'] : 
            sentence_result = " ".join(sentence)
            sentence_result = sentence_result.split(" : ",maxsplit = 2)[-1]
            sentence_result = re.sub(pattern = r"\s\.", repl = ".", string = sentence_result)
            sentence_result = re.sub(pattern = r"\s,\s", repl = ", ", string = sentence_result)
            sentence_result = re.sub(pattern = r"\s\?", repl = "?", string = sentence_result)
            sentence_result = re.sub(pattern = r"\s!", repl = "!", string = sentence_result)
            sentence_result = re.sub(pattern = r" \( ", repl = r"\(", string = sentence_result)
            sentence_result = re.sub(pattern = r" \) ", repl = r"\)", string = sentence_result)
            sentence_result = re.sub(r'\b(\w+)\s+\1\b', r'\1', sentence_result)
            sentence_result = re.sub(r'\s\)', r"\)", sentence_result)
            sentence_result = re.sub(r'\\+', "", sentence_result)
            clean_article_list[-1].append(sentence_result.lower())
        sentence_result = "" 
        for sentence in file['clean_summary'] : 
            sentence_result=" ".join(sentence)
            sentence_result = re.sub(pattern = r"\s\.", repl = ".", string = sentence_result)
            sentence_result = re.sub(pattern = r"\s,\s", repl = ", ", string = sentence_result)
            sentence_result = re.sub(pattern = r"\s\?", repl = "?", string = sentence_result)
            sentence_result = re.sub(pattern = r"\s!", repl = "!", string = sentence_result)
            sentence_result = re.sub(r'\b(\w+)\s+\1\b', r'\1', sentence_result)
            sentence_result = re.sub(r'\s\)', r"\)", sentence_result)
            sentence_result = re.sub(r'\\+', "", sentence_result)
            clean_summary_list[-1].append(sentence_result.lower())
        kunci_jawaban.append(file["extractive_summary"])


In [194]:
rouge_eval = rouge_scorer.RougeScorer(rouge_types=["rouge1","rouge2","rougeL"])

In [195]:
rouge_list = []

for index in range(len(clean_article_list)): 
    article_scores = []  # To hold scores for each summary and context comparison
    for summary in clean_summary_list[index]:  # Iterate over summaries for this article
        summary_scores = []  # To hold scores for each context within the article
        for context in clean_article_list[index]:  # Iterate over contexts for this article
            scoring = rouge_eval.score(summary, context)  # Get ROUGE score
            mean_score = mean([scoring["rouge1"].fmeasure, scoring["rouge2"].fmeasure, scoring["rougeL"].fmeasure])
            # mean_score = scoring["rouge1"].fmeasure
            summary_scores.append(mean_score)  # Append mean score for this context-summary pair
        article_scores.append(summary_scores)  # Append the list of summary-context scores
    rouge_list.append(article_scores)  # Append the article-level list of scores

In [196]:
np.argmax(np.array(rouge_list[0]),axis = -1)

array([8, 7])

In [197]:
extractive_summary_mine = []
for child_list in rouge_list : 
    extractive_summary_mine.append([])
    for angka in np.argmax(np.array(child_list),axis = -1) : 
        extractive_summary_mine[-1].append(int(angka))

In [208]:
clean_summary_list[100]

['kereta eksekutif dari surabaya yang tiba di stasiun gambir jakarta mengalami keterlambatan akibat tanah longsor.',
 'jalur kereta yang menuju surabaya dialihkan melewati jalur alternatif.']

In [207]:
extractive_summary_mine[100]

[0, 14]

In [206]:
kunci_jawaban[100]

[1, 14, 6]